In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import*
from qiskit.visualization import plot_histogram
from math import gcd
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from numpy.random import randint
import pandas as pd
from fractions import Fraction
def c_amod15(a, power):
    qc = QuantumCircuit(2)        
    for iteration in range(power):
        qc.swap(0,1)
        for q in range(2):
            qc.x(q)
    qc = qc.to_gate()            # turning this circuit into gate and plug with shor algorithm
    qc.name = "%i^%i mod 15" % (a, power)
    c_qc = qc.control()
    return c_qc
n = 2
a = 4
def qft(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT"
    return qc
qc = QuantumCircuit(n + 2, n)
for q in range(n):
    qc.h(q)
#qc.x(1+n)
for q in range(n):
    qc.append(c_amod15(a, 2**q), 
             [q] + [i+n for i in range(2)])
qc.draw(initial_state=True,output='mpl')
qc.barrier()
qc.append(qft(n), range(n))
qc.measure(range(n), range(n)) 
qc.draw(initial_state=True,output='mpl')
sim=Aer.get_backend('qasm_simulator')
result=execute(qc,backend=sim,shots=3000).result()
plot_histogram(result.get_counts(qc))

from qiskit import IBMQ
from qiskit import*
IBMQ.load_account()
provider =IBMQ.get_provider(‘ibm-q’)
qccmp =provider.get_backend(‘ibmq_manila’)
job = execute(qc, backend=qccmp, shots = 1000)
from qiskit.tools.monitor import job_monitor
job_monitor(job)
result = job.result()
plot_histogram(result.get_counts(qc))
